In [1]:
# tensorflow imports
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Input, Activation, AveragePooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAvgPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal, GlorotNormal
from tensorflow.keras.utils import plot_model
from scipy.ndimage.interpolation import rotate
from scipy import signal
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import CSVLogger

from tqdm import tqdm
import numpy as np
import os
import pywt
from matplotlib import pyplot as plt
from pywt._doc_utils import wavedec2_keys, draw_2d_wp_basis
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')


In [3]:
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
# a filter
filter1 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [-1,-2,-3], [1,2,3]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])
filtera1 = rotate(filter1, angle=90)
filtera2 = rotate(filter1, angle=180)
filtera3 = rotate(filter1, angle=270)


# d filter
filter2 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [-1,-2,-3],[2,4,6],[-1,-2,-3], [0,0,0]],
                    [[0,0,0], [2,4,6],[-4,-8,-12], [2,4,6],[0,0,0]],
                    [[0,0,0], [-1,-2,-3], [2,4,6], [-1,-2,-3], [0,0,0]],
                    [[0,0,0],[0,0,0],[0,0,0], [0,0,0], [0,0,0]]])


# b filter
fitler3 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [1,2,3], [-2,-4,-6], [1,2,3], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])
filterb1 =  signal.convolve(filter2, filter2, mode='same')
filterb2 =  signal.convolve(filter2, filter2, mode='same')
filterb3 =  signal.convolve(filter2, filter2, mode='same')

# c filter
filter4 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]], [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]], [
    [0,0,0], [1,2,3],[-3,-6,-9],[3,6,9],[-1,-2,-3]], [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])

#this is not filter
filter4RDT = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [1,2,3], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [-3,-6,-9], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [3,6,9], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [-1,-2,-3], [0,0,0], [0,0,0]]])


filterc1 =  signal.convolve(filter4, filter4, mode='same')
filterc2 =  signal.convolve(filter4.T, filter4.T, mode='same').reshape(5,5,3)
filterc3 =  signal.convolve(filter4RDT, filter4RDT, mode='same').reshape(5,5,3)

#e filter
filter5 = np.array([[[-1,-2,-3], [2,4,6], [-2,-4,-6], [2,4,6], [-1,-2,-3]],
                    [[2,4,6], [-6,-12,-18], [-8,-16,-24], [-6,-12,-18], [2,4,6]],
                   [[-2,-4,-6], [8,16,24], [-12,-24,-36], [8,16,24], [2,4,6]],
                    [[2,4,6], [-6,-12,-18], [8,16,24], [-6,-12,-18], [2,4,6]],
                    [[-1,-2,-3], [2,4,6], [-2,-4,-6], [2,4,6], [-1,-2,-3]]])

filter_list = [filter1.tolist(), filtera1.tolist(), filtera2.tolist(), filtera3.tolist(), filter2.tolist(),\
               fitler3.tolist(), filterb1.tolist(), filterb2.tolist(), filterb3.tolist(),\
               filter4.tolist(), filterc1.tolist(), filterc2.tolist(), filterc3.tolist(), filter5.tolist()]

In [5]:
np.array([filter1,filtera1, filtera2, filtera3, filter2, fitler3, filterb1, filterb2, filterb3, filter4, filterc1]).shape

(11, 5, 5, 3)

In [6]:
%load_ext tensorboard

In [7]:
Inp = Input(shape = (64, 64, 3))
x = Conv2D(14, (5,5), padding = 'same', name = 'conv2d_1')(Inp)
x = Conv2D(16, (5,5), kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_2')(x)
x = ZeroPadding2D((1,1))(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(32, (5,5), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_3')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_4')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (1,1), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_5')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(256, (1,1), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_6')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = AveragePooling2D((2,2))(x)
x = Flatten()(x)
out = Dense(1, activation = 'sigmoid',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'ouput')(x)

model = Model(inputs = [Inp], outputs = [out])
model.summary()

#plot_model(model, show_shapes =True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 14)        1064      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 16)        5616      
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 62, 62, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 62, 62, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0     

In [9]:
try:
    os.mkdir('../models/logs')
except Exception as e:
    print(e)

In [10]:
try:
    os.mkdir('../models/weights')
except Exception as e:
    print(e)

In [12]:
try:
    os.mkdir('../models/checkpoints')
except Exception as e:
    print(e)

In [8]:
csv_logger = CSVLogger("../models/weights/tanh_history_log2.csv", append=True)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
log_dir = r"..\\models\\logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
filepath = '../models/checkpoints/model-tanh-try2.h5';
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

In [10]:
earlyStopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [11]:
layer = model.get_layer('conv2d_1')
layer.set_weights([np.array(filter_list).astype(np.float32).reshape(5,5,3,14), np.zeros(14,)])

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255.0,shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1/255.0)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '../input/patches/train/',  # This is the source directory for training images
        target_size=(64, 64),  # All images will be resized to 64x64
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        '../input/patches/validation/',  # This is the source directory for validation images
        target_size=(64, 64),  # All images will be resized to 64x64
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 3751091 images belonging to 2 classes.
Found 537266 images belonging to 2 classes.


In [17]:
train_generator.class_indices

{'fake': 0, 'pristine': 1}

In [18]:
history = model.fit(
      train_generator,  
      epochs=100,
    validation_data = validation_generator,
      verbose=1,
      callbacks=[tensorboard_callback, checkpoint, earlyStopping, csv_logger])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 58611 steps, validate for 8395 steps
Epoch 1/100
58610/58611 [============================>.] - ETA: 0s - loss: 0.4955 - accuracy: 0.7620
Epoch 00001: val_loss improved from inf to 0.47806, saving model to ../models/checkpoints/model-tanh-try2.h5
58611/58611 [==============================] - 5754s 98ms/step - loss: 0.4955 - accuracy: 0.7620 - val_loss: 0.4781 - val_accuracy: 0.7610
Epoch 2/100
58610/58611 [============================>.] - ETA: 0s - loss: 0.4303 - accuracy: 0.8016
Epoch 00002: val_loss improved from 0.47806 to 0.41553, saving model to ../models/checkpoints/model-tanh-try2.h5
58611/58611 [==============================] - 5780s 99ms/step - loss: 0.4303 - accuracy: 0.8016 - val_loss: 0.4155 - val_accuracy: 0.8083
Epoch 3/100
58610/58611 [============================>.] - ETA: 0s - loss: 0.4038 - accuracy: 0.8162
Epoch 00003: val_loss improved from 0.41553 to 0.39746, saving model to ../models/checkpoints/model-

In [20]:
%tensorboard --logdir ../models

Reusing TensorBoard on port 6006 (pid 30260), started 0:03:07 ago. (Use '!kill 30260' to kill it.)